In [1]:
!nvidia-smi

Mon Nov 10 12:51:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install Grounding DINO 🦕

In [3]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .
!pip install -q roboflow

/content
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 463, done.
remote: Total 463 (delta 0), reused 0 (delta 0), pack-reused 463 (from 1)
Receiving objects: 100% (463/463), 12.91 MiB | 34.88 MiB/s, done.
Resolving deltas: 100% (221/221), done.
/content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 25.0 MB/s eta 0:00:00
    error: subprocess-exited-with-error
    
    × python setup.py develop did not run successfully.
    │ exit code: 1
    ╰─> See above for output.
    
    note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× python setup.py develop did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is like

In [4]:
import os

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


## Download Grounding DINO Weights 🏋️

In [5]:
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

/content
/content/weights


In [6]:
import os

WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

/content/weights/groundingdino_swint_ogc.pth ; exist: True


## Download Example Data

In [7]:
%cd {HOME}
!mkdir {HOME}/data
%cd {HOME}/data

!wget -q https://media.roboflow.com/notebooks/examples/dog.jpeg
!wget -q https://media.roboflow.com/notebooks/examples/dog-2.jpeg
!wget -q https://media.roboflow.com/notebooks/examples/dog-3.jpeg
!wget -q https://media.roboflow.com/notebooks/examples/dog-4.jpeg

/content
/content/data


## Load Grounding DINO Model

In [8]:
%cd {HOME}/GroundingDINO

from groundingdino.util.inference import load_model, load_image, predict, annotate

model = load_model(CONFIG_PATH, WEIGHTS_PATH)

/content/GroundingDINO


final text_encoder_type: bert-base-uncased


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

## Grounding DINO

In [12]:
import requests

import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection, infer_device

model_id = "IDEA-Research/grounding-dino-tiny"
device = infer_device()

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)

image = Image.open('/content/download (copy 2).png')
# Check for cats and remote controls
text_labels = [["a cat", "a remote control"]]

inputs = processor(images=image, text=text_labels, return_tensors="pt").to(model.device)
with torch.no_grad():
    outputs = model(**inputs)

results = processor.post_process_grounded_object_detection(
    outputs,
    inputs.input_ids,
    threshold=0.4,
    text_threshold=0.3,
    target_sizes=[image.size[::-1]]
)

result = results[0]
for box, score, labels in zip(result["boxes"], result["scores"], result["labels"]):
    box = [round(x, 2) for x in box.tolist()]
    print(f"Detected {labels} with confidence {round(score.item(), 3)} at location {box}")

Detected a cat with confidence 0.406 at location [0.04, -0.1, 475.75, 511.86]


In [13]:
# سلول ۲: بارگذاری مدل‌ها و تعریف توابع

import torch
import os
import numpy as np
from PIL import Image
import cv2
import json
import requests
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
from diffusers import StableDiffusionPipeline

# --- ۱. تنظیمات و دستگاه ---
HUGGINGFACE_DINO_MODEL_ID = "IDEA-Research/grounding-dino-tiny"
SD_MODEL_ID = "runwayml/stable-diffusion-v1-5"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"🚀 دستگاه مورد استفاده: {device}")

# --- ۲. بارگذاری Stable Diffusion ---
print("🚀 در حال بارگذاری Stable Diffusion...")
# استفاده از torch.float16 برای کاهش مصرف حافظه GPU
pipe_sd = StableDiffusionPipeline.from_pretrained(SD_MODEL_ID, torch_dtype=torch.float16)
pipe_sd = pipe_sd.to(device)
print("✅ Stable Diffusion بارگذاری شد.")

# --- ۳. بارگذاری Grounding DINO Tiny ---
print("🚀 در حال بارگذاری Grounding DINO Tiny...")
dino_processor = AutoProcessor.from_pretrained(HUGGINGFACE_DINO_MODEL_ID)
dino_model = AutoModelForZeroShotObjectDetection.from_pretrained(HUGGINGFACE_DINO_MODEL_ID).to(device)
print("✅ Grounding DINO Tiny بارگذاری شد.")

# --- ۴. توابع کمکی ---

def to_yolo_format(boxes_pixel, class_id, img_w, img_h):
    """تبدیل مختصات پیکسلی (xmin, ymin, xmax, ymax) به فرمت YOLO (نرمالیزه شده)"""
    yolo_annotations = []
    for xmin, ymin, xmax, ymax in boxes_pixel:
        # مختصات مرکز
        x_center = ((xmin + xmax) / 2) / img_w
        y_center = ((ymin + ymax) / 2) / img_h
        # عرض و ارتفاع
        width = (xmax - xmin) / img_w
        height = (ymax - ymin) / img_h
        yolo_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
    return "\n".join(yolo_annotations)

# --- ۵. تابع اصلی تولید پکیج YOLO ---

def generate_yolo_package(
    customer_prompt: str,           # پرامپت Stable Diffusion برای تولید تصویر
    detection_target_text: str,     # پرامپت تشخیص Grounding DINO (مثلاً: "red cube . green cube")
    class_name: str,                # نام کلاس برای فایل‌ها (مثلاً: "cube")
    class_id: int,                  # ID کلاس در YOLO (مثلاً: 0)
    seed: int = 42,
    output_dir: str = "YOLO_Generated_Dataset",
    box_threshold: float = 0.4,
    text_threshold: float = 0.3
):
    """
    تولید یک تصویر توسط Stable Diffusion، تشخیص شیء با Grounding DINO و ذخیره در فرمت YOLO.
    """
    os.makedirs(output_dir, exist_ok=True)

    # 1. تولید تصویر (Stable Diffusion)
    generator = torch.Generator(device).manual_seed(seed)
    with torch.no_grad():
        result = pipe_sd(customer_prompt, num_inference_steps=50, guidance_scale=7.5, generator=generator)
    generated_image_pil = result.images[0]
    img_w, img_h = generated_image_pil.size

    # 2. تشخیص آبجکت (Grounding DINO)
    # Grounding DINO از پرامپت چندگانه در یک لیست پشتیبانی می‌کند.
    text_labels = [[t.strip() for t in detection_target_text.split('.') if t.strip()]]

    inputs = dino_processor(images=generated_image_pil, text=text_labels, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = dino_model(**inputs)

    results = dino_processor.post_process_grounded_object_detection(
        outputs,
        inputs.input_ids,
        threshold=box_threshold,
        text_threshold=text_threshold,
        target_sizes=[generated_image_pil.size[::-1]]
    )
    result = results[0]

    boxes_tensor = result["boxes"]
    scores_tensor = result["scores"]

    if len(boxes_tensor) == 0:
        print(f"⚠️ اخطار: هیچ آبجکتی برای '{detection_target_text}' تشخیص داده نشد. (Seed: {seed})")
        return None # اگر چیزی پیدا نشد، خروجی نده.

    boxes_pixel = boxes_tensor.cpu().numpy().astype(int)
    scores_np = scores_tensor.cpu().numpy()

    # 3. تولید Annotation و Metadata
    yolo_annotations = to_yolo_format(boxes_pixel, class_id, img_w, img_h)

    metadata = {
        "Prompt_Diffusion": customer_prompt, "Detection_Target_Text": detection_target_text,
        "Class_Name": class_name, "Class_ID": class_id, "Seed": seed,
        "Detected_Objects_Count": len(boxes_pixel),
        "Detected_Confidences": [f"{l:.2f}" for l in scores_np.tolist()]
    }

    # 4. ذخیره‌سازی نهایی
    asset_base_name = f"asset_{seed}_{class_name.replace(' ', '_')}"
    output_path = os.path.join(output_dir, f"{asset_base_name}.png")

    generated_image_pil.save(output_path)
    with open(os.path.join(output_dir, f"{asset_base_name}.txt"), 'w') as f: f.write(yolo_annotations)
    with open(os.path.join(output_dir, f"{asset_base_name}_metadata.json"), 'w') as f: json.dump(metadata, f, indent=4)

    print(f"✅ پکیج YOLO برای '{class_name}' با Seed={seed} ساخته شد. مسیر: {output_path}")
    return output_path

🚀 دستگاه مورد استفاده: cuda
🚀 در حال بارگذاری Stable Diffusion...


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


✅ Stable Diffusion بارگذاری شد.
🚀 در حال بارگذاری Grounding DINO Tiny...
✅ Grounding DINO Tiny بارگذاری شد.


In [16]:
# سلول ۳: اجرای نهایی

generate_yolo_package(
    customer_prompt="a photo of tow cat with distance",
    detection_target_text="cat",
    class_name="cat",
    class_id=0,
    seed=100,
    output_dir="cat_YOLO_Dataset"
)

# یک مثال دیگر با پرامپت پیچیده‌تر و Seed متفاوت
generate_yolo_package(
    customer_prompt="A photo of five dog with distance of them",
    detection_target_text="dog",
    class_name="dog",
    class_id=1,
    seed=200,
    output_dir="dog__YOLO_Dataset"
)

print("\n---")
print("🎉 تولید دیتاست اولیه با موفقیت انجام شد. پوشه‌های خروجی را بررسی کنید.")

  0%|          | 0/50 [00:00<?, ?it/s]

✅ پکیج YOLO برای 'cat' با Seed=100 ساخته شد. مسیر: cat_YOLO_Dataset/asset_100_cat.png


  0%|          | 0/50 [00:00<?, ?it/s]

✅ پکیج YOLO برای 'dog' با Seed=200 ساخته شد. مسیر: dog__YOLO_Dataset/asset_200_dog.png

---
🎉 تولید دیتاست اولیه با موفقیت انجام شد. پوشه‌های خروجی را بررسی کنید.
